## Imports

In [1]:
from utensor_cgen.api.export import tflm_keras_export

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

tf.keras.backend.clear_session()

## Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

[tf.keras.datasets.mnist](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data) returns: ***Tuple of Numpy arrays: (x_train, y_train), (x_test, y_test).***

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Use tf.data to batch and shuffle the dataset.

from_tensor_slices

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Defining the Model

In [5]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(8, 3, activation='relu')
    self.pool = MaxPooling2D()
    self.flatten = Flatten()
    self.d1 = Dense(16, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x0 = self.pool(x)
    x1 = self.conv1(x0)
    x2 = self.pool(x1)
    x3 = self.flatten(x2)
    x4 = self.d1(x3)
    return self.d2(x4)

model = MyModel()

Choose an optimizer and loss function for training:

In [6]:
# see
# https://github.com/tensorflow/tensorflow/blob/e5bf8de410005de06a7ff5393fafdf832ef1d4ad/tensorflow/python/keras/utils/layer_utils.py#L224-L228
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer
def print_model_summary(model, input_shape):
  input_shape_nobatch = input_shape[1:]
  model.build(input_shape)
  inputs = tf.keras.Input(shape=input_shape_nobatch)
  
  if not hasattr(model, 'call'):
      raise AttributeError("User should define 'call' method in sub-class model!")
  
  _ = model.call(inputs)
  #model.layers[0].output_shape
  #model.layers[0].count_params()
  #
  model.summary()

print_model_summary(model, x_test.shape)

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 12, 12, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                4624      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                170       
Total params: 4,874
Trainable params: 4,874
Non-trainable params: 0
_________________________________________________________________


In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [10]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Training step

In [11]:
EPOCHS = 15

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.459749698638916, Accuracy: 86.40333557128906, Test Loss: 0.18603216111660004, Test Accuracy: 94.27000427246094
Epoch 2, Loss: 0.1707976907491684, Accuracy: 94.72833251953125, Test Loss: 0.13616280257701874, Test Accuracy: 95.6300048828125
Epoch 3, Loss: 0.1336827278137207, Accuracy: 95.79166412353516, Test Loss: 0.10996600985527039, Test Accuracy: 96.49000549316406
Epoch 4, Loss: 0.11653120815753937, Accuracy: 96.35499572753906, Test Loss: 0.10350492596626282, Test Accuracy: 96.62000274

## Testing the Model

In [12]:
test_image, test_label = next(test_ds.__iter__())
test_image = tf.slice(test_image, [0,0,0,0], [1,-1,-1,-1])
prediction = model(test_image, training=False)
print("label: ", test_label[0])
print("pred:  ", tf.math.argmax(prediction[0]))

label:  tf.Tensor(7, shape=(), dtype=uint8)
pred:   tf.Tensor(7, shape=(), dtype=int64)


In [13]:
model.save('saved_model/test_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/test_model/assets


## Exporting to uTensor

A sample generator for the offline-quantizer to estimate the optimal quantization ranges for all activations.

In [14]:
# representative data function
num_calibration_steps = 128
calibration_dtype = tf.float32

def representative_dataset_gen():
    for _ in range(num_calibration_steps):
        rand_idx = np.random.randint(0, x_test.shape[0]-1)
        sample = x_test[rand_idx]
        sample = sample[tf.newaxis, ...]
        sample = tf.cast(sample, dtype=calibration_dtype)
        yield [sample]

One-liner uTensor export API

In [15]:
tflm_keras_export(
    'saved_model/test_model',
    representive_dataset=representative_dataset_gen,
    model_name='my_model',
    target='utensor',
)

[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING quantize.py <module> @ 12] trying to import deprecated quantization transformer
[INFO transformer.py transform @ 23] Transforming graph: my_model
[INFO transformer.py transform @ 24] Transform pipeline: dropout(name_pattern=r'(dropout[_\w\d]*)/.*') -> linear_reorder -> inline -> biasAdd -> remove_id_op -> fake_gather_v2 -> refcnt
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING ns_transformer.py transform @ 243] enabling fake_gather_v2 will force replacing GatherV2 w

In [16]:
!pwd

/Users/neiltan/code/ut/utensor-mnist-demo/tensorflow-models
